In [1]:
#import modules
import json
from binance.client import Client
from dotenv import load_dotenv
from envs import env
import re
import os
import matplotlib.pyplot as plt

In [2]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

In [3]:
load_dotenv('../.envall')
try:
    liveVolume=env("liveVolume")
    apiSecret=env('apiSecret')
    apiKey=env('apiKey')
    baseCurrency=env('baseCurrency')
except KeyError:
    print("No env variables set.")
    sys.exit(1)
client = Client(apiKey, apiSecret, {'timeout':600})

In [4]:
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750

In [5]:
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

In [6]:
def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [7]:
get_all_binance("BNBETH", "1m", save = True)

All caught up..!


,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-09 09:00:00,0.00081993,0.00081993,0.00081993,0.00081993,300.00000000,1502269259999,0.24597900,1,300.00000000,0.24597900,101104096.09190453
2017-08-09 09:01:00,0.00081993,0.00082284,0.00081993,0.00082046,11654.00000000,1502269319999,9.56095260,10,8387.00000000,6.88096275,101104096.09190453
2017-08-09 09:02:00,0.00081875,0.00082284,0.00081875,0.00082284,8781.00000000,1502269379999,7.20909980,5,8781.00000000,7.20909980,101104096.09190453
2017-08-09 09:03:00,0.00081974,0.00082884,0.00081974,0.00082232,21693.00000000,1502269439999,17.85982611,11,20314.00000000,16.72940465,101104096.09190453
2017-08-09 09:04:00,0.00082236,0.00082270,0.00082127,0.00082270,5895.00000000,1502269499999,4.84898825,9,3340.00000000,2.74781800,101104096.09190453
...,...,...,...,...,...,...,...,...,...,...,...
2021-07-19 19:47:00,0.15520000,0.15525000,0.15512000,0.15515000,4.36300000,1626724079999,0.67691619,7,3.27300000,0.50782995,0
2021-07-19 19:48:00,0.15510000,0.15510000,0.15498000,0.15498000,45.93100000,1626724139999,7.11857818,5,0.19600000,0.03039960,0
2021-07-19 19:49:00,0.15506000,0.15514000,0.15499000,0.15505000,18.18700000,1626724199999,2.82017218,18,9.11400000,1.41337817,0
